## Installing the necessary packages

In [ ]:
!pip install transformers
!pip install torch
!pip install numpy

In [1]:
from datetime import timedelta
import time

## The code







We use transformers library from HuggingFace to load a pretrained gpt2 model that uses beam search in which we will add our constraints ausing a deterministic automata.

In [2]:
from transformers import AutoTokenizer, AutoModelForCausalLM

tokenizer = AutoTokenizer.from_pretrained('gpt2')
model = AutoModelForCausalLM.from_pretrained("gpt2")

C:\Users\Buse\anaconda3\envs\afltproj\lib\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## Automata

In [3]:
from rayuela.base.semiring import Boolean, Real, Tropical, \
    String, Integer, Rational
from rayuela.base.symbol import Sym, ε
from rayuela.fsa.fsa import FSA
from rayuela.fsa.state import State
import numpy as np
from string import ascii_lowercase, ascii_uppercase

This first automata we use as an example will only accept strings that have the sentence structure: a... -> b... -> c... -> a... This means once a sentence has a word starting with an "a" it should have a word starting with "b" next and so on. (This blog post has helped with the idea of coming up with this constraint: https://medium.com/towards-data-science/the-power-of-constrained-language-models-cf63b65a035d)

In [4]:
# You can visualize this FSA to convince yourself it works
def three_consecutive_word_FSA(letter_a = ["a","A"], letter_b = ["b","B"], letter_c = ["c","C"]):
    fsa = FSA(Boolean)
    space = " "
    special_chars = letter_a + letter_b + letter_c
    
    fsa.set_I(State(0))
    for c in ascii_lowercase + ascii_uppercase + space:
        if c in letter_a:
            fsa.add_arc(State(0), Sym(c), State(1))
            fsa.add_arc(State(6), Sym(c), State(1))
            fsa.add_arc(State(7), Sym(c), State(7))
            fsa.add_arc(State(1), Sym(c), State(1))
            fsa.add_arc(State(3), Sym(c), State(3))
            fsa.add_arc(State(5), Sym(c), State(5))
            fsa.add_arc(State(2), Sym(c), State(8))
            fsa.add_arc(State(4), Sym(c), State(8))
        elif c in letter_b:
            fsa.add_arc(State(2), Sym(c), State(3))
            fsa.add_arc(State(0), Sym(c), State(7))
            fsa.add_arc(State(7), Sym(c), State(7))
            fsa.add_arc(State(1), Sym(c), State(1))
            fsa.add_arc(State(3), Sym(c), State(3))
            fsa.add_arc(State(5), Sym(c), State(5))
            fsa.add_arc(State(4), Sym(c), State(8))
            fsa.add_arc(State(6), Sym(c), State(8))
        elif c in letter_c:
            fsa.add_arc(State(4), Sym(c), State(5))
            fsa.add_arc(State(0), Sym(c), State(7))
            fsa.add_arc(State(7), Sym(c), State(7))
            fsa.add_arc(State(1), Sym(c), State(1))
            fsa.add_arc(State(3), Sym(c), State(3))
            fsa.add_arc(State(5), Sym(c), State(5))
            fsa.add_arc(State(2), Sym(c), State(8))
            fsa.add_arc(State(6), Sym(c), State(8))
        elif c == space:
            fsa.add_arc(State(1), Sym(c), State(2))
            fsa.add_arc(State(3), Sym(c), State(4))
            fsa.add_arc(State(5), Sym(c), State(6))
            fsa.add_arc(State(7), Sym(c), State(0))
            fsa.add_arc(State(0), Sym(c), State(7))
            fsa.add_arc(State(2), Sym(c), State(8))
            fsa.add_arc(State(4), Sym(c), State(8))
            fsa.add_arc(State(6), Sym(c), State(8))
        else:
            fsa.add_arc(State(0), Sym(c), State(7))
            fsa.add_arc(State(7), Sym(c), State(7))
            fsa.add_arc(State(1), Sym(c), State(1))
            fsa.add_arc(State(3), Sym(c), State(3))
            fsa.add_arc(State(5), Sym(c), State(5))
            fsa.add_arc(State(2), Sym(c), State(8))
            fsa.add_arc(State(4), Sym(c), State(8))
            fsa.add_arc(State(6), Sym(c), State(8))
    
    fsa.set_F(State(0))
    fsa.set_F(State(1))
    fsa.set_F(State(2))
    fsa.set_F(State(3))
    fsa.set_F(State(4))
    fsa.set_F(State(5))
    fsa.set_F(State(6))
    fsa.set_F(State(7))
    
    return fsa

In [5]:
three_consecutive_word_FSA()

WFSA(9 states, <class 'rayuela.base.semiring.Boolean'>)

This automata accepts even length strings.

In [6]:
def even_FSA():
    fsa = FSA(Boolean)
    space = " "
    nl = "\n"
    
    fsa.set_I(State(0))
    for c in ascii_lowercase + ascii_uppercase:
        fsa.add_arc(State(0), Sym(c), State(1))
        fsa.add_arc(State(1), Sym(c), State(0))
    for c in space + nl:
        fsa.add_arc(State(0), Sym(c), State(0))
        fsa.add_arc(State(1), Sym(c), State(1))
    fsa.set_F(State(0))
    
    return fsa

In [7]:
even_FSA()

WFSA(2 states, <class 'rayuela.base.semiring.Boolean'>)

This automata will enforce all lines to have 4 words and the last word to rhyme. This is a manual construction of the two constraints we enforce: having 4 words each line and having rhyme at each line.

In [8]:
def word_count_rhyme_FSA_visualization(n=4,first_letter=["l","L"], second_letter=["y","Y"]):
    fsa = FSA(Boolean)
    space = " "
    sp_sym = Sym("sp")
    nl = "\n"
    nl_sym = Sym("nl")
    fsa.set_I(State(0))

    #ascii_lowercase + ascii_uppercase + space + nl
    for c in ascii_lowercase + ascii_uppercase + space + nl:
        if c in first_letter:
            fsa.add_arc(State(0), Sym(c), State(0))
            fsa.add_arc(State(1), Sym(c), State(2))
            fsa.add_arc(State(2), Sym(c), State(2))
            fsa.add_arc(State(3), Sym(c), State(4))
            fsa.add_arc(State(4), Sym(c), State(4))
            for i in range(n-2):
                fsa.add_arc(State(i+5), Sym(c), State(i+5))
        elif c in second_letter:
            fsa.add_arc(State(0), Sym(c), State(0))
            fsa.add_arc(State(1), Sym(c), State(1))
            fsa.add_arc(State(2), Sym(c), State(3))
            fsa.add_arc(State(3), Sym(c), State(4))
            fsa.add_arc(State(4), Sym(c), State(4))
            for i in range(n-2):
                fsa.add_arc(State(i+5), Sym(c), State(i+5))
        elif c == space:
            fsa.add_arc(State(1), sp_sym, State(4))
            fsa.add_arc(State(2), sp_sym, State(4))
            fsa.add_arc(State(3), sp_sym, State(4))
            fsa.add_arc(State(4), sp_sym, State(4))
            
            fsa.add_arc(State(0), sp_sym, State(5))
            for i in range(n-3):
                fsa.add_arc(State(i+5), sp_sym, State(i+6))
            fsa.add_arc(State(n+2), sp_sym, State(1))
        elif c == nl:
            fsa.add_arc(State(0), nl_sym, State(4))
            fsa.add_arc(State(1), nl_sym, State(4))
            fsa.add_arc(State(2), nl_sym, State(4))
            fsa.add_arc(State(3), nl_sym, State(0))
            fsa.add_arc(State(4), nl_sym, State(4))
            for i in range(n-2):
                fsa.add_arc(State(i+5), nl_sym, State(4))
        else:
            fsa.add_arc(State(0), Sym(c), State(0))
            fsa.add_arc(State(1), Sym(c), State(1))
            fsa.add_arc(State(2), Sym(c), State(1))
            fsa.add_arc(State(3), Sym(c), State(4))
            fsa.add_arc(State(4), Sym(c), State(4))
            for i in range(n-2):
                fsa.add_arc(State(i+5), Sym(c), State(i+5))

    fsa.set_F(State(0))
    for i in range(n-2):
        fsa.set_F(State(i+5))
    fsa.set_F(State(3))
    return fsa

In [9]:
def word_count_rhyme_FSA(n=4,first_letter=["l","L"], second_letter=["y","Y"]):
    fsa = FSA(Boolean)
    space = " "
    nl = "\n"
    fsa.set_I(State(0))

    #ascii_lowercase + ascii_uppercase + space + nl
    for c in ascii_lowercase + ascii_uppercase + space + nl:
        if c in first_letter:
            fsa.add_arc(State(0), Sym(c), State(0))
            fsa.add_arc(State(1), Sym(c), State(2))
            fsa.add_arc(State(2), Sym(c), State(2))
            fsa.add_arc(State(3), Sym(c), State(4))
            fsa.add_arc(State(4), Sym(c), State(4))
            for i in range(n-2):
                fsa.add_arc(State(i+5), Sym(c), State(i+5))
        elif c in second_letter:
            fsa.add_arc(State(0), Sym(c), State(0))
            fsa.add_arc(State(1), Sym(c), State(1))
            fsa.add_arc(State(2), Sym(c), State(3))
            fsa.add_arc(State(3), Sym(c), State(4))
            fsa.add_arc(State(4), Sym(c), State(4))
            for i in range(n-2):
                fsa.add_arc(State(i+5), Sym(c), State(i+5))
        elif c == space:
            fsa.add_arc(State(1), Sym(c), State(4))
            fsa.add_arc(State(2), Sym(c), State(4))
            fsa.add_arc(State(3), Sym(c), State(4))
            fsa.add_arc(State(4), Sym(c), State(4))
            fsa.add_arc(State(0), Sym(c), State(5))
            for i in range(n-3):
                fsa.add_arc(State(i+5), Sym(c), State(i+6))
            fsa.add_arc(State(n+2), Sym(c), State(1))
        elif c == nl:
            fsa.add_arc(State(0), Sym(c), State(4))
            fsa.add_arc(State(1), Sym(c), State(4))
            fsa.add_arc(State(2), Sym(c), State(4))
            fsa.add_arc(State(3), Sym(c), State(0))
            fsa.add_arc(State(4), Sym(c), State(4))
            for i in range(n-2):
                fsa.add_arc(State(i+5), Sym(c), State(4))
        else:
            fsa.add_arc(State(0), Sym(c), State(0))
            fsa.add_arc(State(1), Sym(c), State(1))
            fsa.add_arc(State(2), Sym(c), State(1))
            fsa.add_arc(State(3), Sym(c), State(4))
            fsa.add_arc(State(4), Sym(c), State(4))
            for i in range(n-2):
                fsa.add_arc(State(i+5), Sym(c), State(i+5))

    fsa.set_F(State(0))
    for i in range(n-2):
        fsa.set_F(State(i+5))
    fsa.set_F(State(3))
    return fsa

In [10]:
manual_fsa = word_count_rhyme_FSA(n=5)
word_count_rhyme_FSA_visualization(n=5)

WFSA(8 states, <class 'rayuela.base.semiring.Boolean'>)

## Constraints

This helper function is taken from source code of the model, with this we can set the scored of banned tokens we calculate with our automata to minus infinity.

In [11]:
# src: https://huggingface.co/transformers/v4.1.1/_modules/transformers/generation_logits_process.html

def set_scores_to_inf_for_banned_tokens(scores, banned_tokens):
    """
    Modifies the scores in place by setting the banned token positions to `-inf`. Banned token is expected to be a
    list of list of banned tokens to ban in the format [[batch index, vocabulary position],...

    Args:
        scores: logits distribution of shape (batch size, vocabulary size)
        banned_tokens: list of list of tokens to ban of length (batch_size)
    """
    banned_mask_list = []
    for idx, batch_banned_tokens in enumerate(banned_tokens):
        for token in batch_banned_tokens:
            banned_mask_list.append([idx, token])
    if not banned_mask_list:
        return scores

    banned_mask = torch.LongTensor(banned_mask_list)
    indices = torch.ones(len(banned_mask))

    banned_mask = (
        torch.sparse.LongTensor(banned_mask.t(), indices, scores.size()).to(scores.device).to_dense().bool()
    )
    scores = scores.masked_fill(banned_mask, -float("inf"))
    return scores

Helper function to calculate acceptance of string deterministically.

In [12]:
def accept_deterministic(fsa, string):
    string_chars = list(string)
    
    for q,_ in fsa.I:
        next_state = q
    for ch in string_chars:
        found = False
        if Sym(ch) not in fsa.Sigma:
            return False
        for a, q, _ in fsa.arcs(next_state):
            if a == Sym(ch):
                found = True
                next_state = q
                break
        if not found:
            return False

    return fsa.ρ[next_state] != fsa.R.zero

Logits processor models we define for different constraints

In [13]:
from transformers import LogitsProcessor

This model applies the constraint to each token in the library once then proceeds to use these values in the beam search.

In [14]:
class WordLogits(LogitsProcessor):

    def __init__(self, fsa, vocab):
        
        self.fsa = fsa
        self.keys = np.array(list(vocab.keys()))
        self.values = np.array(list(vocab.values()))
        
        # add token to banned tokens if the fsa does not accept 
        accepted = lambda x: self.fsa.accept_deterministic(x.strip('Ġ '))
        vec_accepted = np.vectorize(accepted)
        indexes = np.where(vec_accepted(self.keys) == False)[0]
        self.banned_values = self.values[indexes]

    def __call__(self, input_ids, scores):
        
        banned_tokens = []
        for beam_index, (beam_input_ids, beam_scores) in enumerate(zip(input_ids, scores)):
            banned_tokens.append(self.banned_values)
        
        # set the scores of all banned tokens over the beams to -inf
        scores = set_scores_to_inf_for_banned_tokens(scores, banned_tokens)
        return scores

This model checks if the automata accepts the string generated in this line, if not it sets the value of that token to minus infinity. Works for deterministic.

In [15]:
class ConsecutiveLogits(LogitsProcessor):
    def __init__(self, fsa, vocab):
        
        self.fsa = fsa
        self.keys = np.array(list(vocab.keys()))
        self.values = np.array(list(vocab.values()))
        
    def __call__(self, input_ids, scores):
        
        banned_tokens = []
        for beam_index, (beam_input_ids, beam_scores) in enumerate(zip(input_ids, scores)):
            
            start = time.time()
            main_beam_string = tokenizer.decode(beam_input_ids)
            print(main_beam_string)
            
            new_line=main_beam_string.split("\n")[-1]
            accepted = lambda x: accept_deterministic(self.fsa, str(new_line + x.replace("Ġ", " ").replace("Ċ", "\n")))
            vec_accepted = np.vectorize(accepted)
            indexes = np.where(vec_accepted(self.keys) == False)[0]

            banned_tokens.append(self.values[indexes])
            
            end = time.time()
            elapsed = (end - start)
            print("Time: ", str(timedelta(seconds=elapsed)))
            start=end
        
        scores = set_scores_to_inf_for_banned_tokens(scores, banned_tokens)            
        return scores

## Results

Sentence generator with the structure: a... -> b... -> c... -> a...

In [8]:
from transformers import (
    BeamSearchScorer,
    LogitsProcessorList,
    StoppingCriteriaList,
    MaxLengthCriteria
)
import torch

# how many beams to track during the Viterbi algorithm
num_beams = 5
# how many beams to return after the algorithm
num_return_beams = 5

# the prompt to continue
prompt = 'My dog is a'

# tokenizing the prompt
prompt_tokenized = tokenizer(prompt, return_tensors='pt' )
prompt_tokenized = prompt_tokenized['input_ids']

# instantiating a BeamSearchScorer
beam_scorer = BeamSearchScorer(
    batch_size = prompt_tokenized.shape[0],
    num_beams = num_beams,
    num_beam_hyps_to_keep = num_return_beams,
    device=model.device
)

# using our automata to enforce the desired sequence
fsa = three_consecutive_word_FSA()
logits_processor = LogitsProcessorList([ConsecutiveLogits(fsa=fsa, vocab=tokenizer.vocab)])

# running beam search using our custom LogitsProcessor
generated = model.beam_search(
    torch.cat([prompt_tokenized] * num_beams),
    beam_scorer,
    logits_processor = logits_processor,
    stopping_criteria=StoppingCriteriaList([MaxLengthCriteria(max_length=12)])
)

# printing the output beams
for index, output_tokenized in enumerate(generated):
  output = tokenizer.decode(output_tokenized)
  print(f'beam {index}: {output}')

My dog is a
Time:  0:00:27.622529
My dog is a
Time:  0:00:26.349971
My dog is a
Time:  0:00:26.396049
My dog is a
Time:  0:00:26.318951
My dog is a
Time:  0:00:27.441226
My dog is a bit
Time:  0:00:35.829037
My dog is a big
Time:  0:00:32.109995
My dog is a beautiful
Time:  0:00:38.295965
My dog is a bitch
Time:  0:00:37.720999
My dog is a baby
Time:  0:00:34.721262
My dog is a big cat
Time:  0:00:37.959998
My dog is a beautiful creature
Time:  0:00:50.009582
My dog is a bit crazy
Time:  0:00:42.146992
My dog is a bit confused
Time:  0:00:45.716012
My dog is a bit clumsy
Time:  0:00:43.874961
My dog is a big cat and
Time:  0:00:44.226213
My dog is a beautiful creature and
Time:  0:00:56.052032
My dog is a bit confused about
Time:  0:01:06.812506
My dog is a bit clumsy and
Time:  0:00:57.446896
My dog is a bit crazy and
Time:  0:01:15.424194
My dog is a bit confused about being
Time:  0:01:03.661999
My dog is a big cat and because
Time:  0:00:54.803559
My dog is a big cat and by
Time:  

Poem generator using the manually constructed fsa.

In [75]:
from transformers import (
    BeamSearchScorer,
    LogitsProcessorList,
    RepetitionPenaltyLogitsProcessor,
    StoppingCriteriaList,
    MaxLengthCriteria
)
import torch

# how many beams to track during the Viterbi algorithm
num_beams = 3
# how many beams to return after the algorithm
num_return_beams = 3

# the prompt to continue
prompt = 'Standing here all lonely'

# tokenizing the prompt
prompt_tokenized = tokenizer(prompt, return_tensors='pt' )
prompt_tokenized = prompt_tokenized['input_ids']

# instantiating a BeamSearchScorer
beam_scorer = BeamSearchScorer(
    batch_size = prompt_tokenized.shape[0],
    num_beams = num_beams,
    num_beam_hyps_to_keep = num_return_beams,
    device=model.device
)

# using our automata to enforce the desired sequence
fsa4 = word_count_rhyme_FSA()
logits_processor = LogitsProcessorList([ConsecutiveLogits(fsa=fsa4, vocab=tokenizer.vocab), RepetitionPenaltyLogitsProcessor(3.0)])

# running beam search using our custom LogitsProcessor
generated = model.beam_search(
    torch.cat([prompt_tokenized] * num_beams),
    beam_scorer,
    logits_processor = logits_processor,
    stopping_criteria=StoppingCriteriaList([MaxLengthCriteria(max_length=100)])
)

# printing the output beams
for index, output_tokenized in enumerate(generated):
  output = tokenizer.decode(output_tokenized)
  print(f'beam {index}: {output}')

Standing here all lonely
Time:  0:00:41.660173
Standing here all lonely
Time:  0:00:40.847996
Standing here all lonely
Time:  0:00:40.623015
Standing here all lonely

Time:  0:00:08.679001
Standing here all lonely

Time:  0:00:08.744037
Standing here all lonely

Time:  0:00:09.223335
Standing here all lonely
I
Time:  0:00:11.674390
Standing here all lonely
The
Time:  0:00:13.773902
Standing here all lonely
A
Time:  0:00:10.873013
Standing here all lonely
I can
Time:  0:00:17.274066
Standing here all lonely
I am
Time:  0:00:16.647019
Standing here all lonely
I don
Time:  0:00:18.268003
Standing here all lonely
I can see
Time:  0:00:23.099589
Standing here all lonely
I can feel
Time:  0:00:23.710000
Standing here all lonely
I am a
Time:  0:00:21.014014
Standing here all lonely
I can see only
Time:  0:00:35.833927
Standing here all lonely
I can see clearly
Time:  0:00:33.832036
Standing here all lonely
I can feel only
Time:  0:00:32.480966
Standing here all lonely
I can see clearly

Time:

Time:  0:00:09.377218
Standing here all lonely
I can see clearly
And I feel deeply
Like a childless butterfly
In the darkness only
My mind is completely

Time:  0:00:09.002039
Standing here all lonely
I can see clearly
And I feel deeply
Like a childless butterfly
In the darkness only
My mind is slowly
But
Time:  0:00:14.237000
Standing here all lonely
I can see clearly
And I feel deeply
Like a childless butterfly
In the darkness only
My mind is slowly
The
Time:  0:00:13.335039
Standing here all lonely
I can see clearly
And I feel deeply
Like a childless butterfly
In the darkness only
My mind is slowly
As
Time:  0:00:12.423999
Standing here all lonely
I can see clearly
And I feel deeply
Like a childless butterfly
In the darkness only
My mind is slowly
As it
Time:  0:00:18.554002
Standing here all lonely
I can see clearly
And I feel deeply
Like a childless butterfly
In the darkness only
My mind is slowly
As if
Time:  0:00:17.358000
Standing here all lonely
I can see clearly
And I feel de

Time:  0:00:08.608995
Standing here all lonely
I can see clearly
And I feel deeply
Like a childless butterfly
In the darkness only
My mind is slowly
But my heart truly
Is filled with melancholy
As if it suddenly
Was
Time:  0:00:13.865039
Standing here all lonely
I can see clearly
And I feel deeply
Like a childless butterfly
In the darkness only
My mind is slowly
But my heart truly
Is filled with melancholy
As if it suddenly
A
Time:  0:00:10.601962
Standing here all lonely
I can see clearly
And I feel deeply
Like a childless butterfly
In the darkness only
My mind is slowly
But my heart truly
Is filled with melancholy
As if it suddenly
It
Time:  0:00:12.855001
Standing here all lonely
I can see clearly
And I feel deeply
Like a childless butterfly
In the darkness only
My mind is slowly
But my heart truly
Is filled with melancholy
As if it suddenly
Was born
Time:  0:00:21.566035
Standing here all lonely
I can see clearly
And I feel deeply
Like a childless butterfly
In the darkness only
My 

Time:  0:00:20.051033
Standing here all lonely
I can see clearly
And I feel deeply
Like a childless butterfly
In the darkness only
My mind is slowly
But my heart truly
Is filled with melancholy
As if it suddenly
Was born of folly
From which nothing really
Has come
Time:  0:00:19.546968
Standing here all lonely
I can see clearly
And I feel deeply
Like a childless butterfly
In the darkness only
My mind is slowly
But my heart truly
Is filled with melancholy
As if it suddenly
Was born of folly
From which nothing really
Can escape me
Time:  0:00:25.349998
Standing here all lonely
I can see clearly
And I feel deeply
Like a childless butterfly
In the darkness only
My mind is slowly
But my heart truly
Is filled with melancholy
As if it suddenly
Was born of folly
From which nothing actually
Can escape me
Time:  0:00:25.751006
Standing here all lonely
I can see clearly
And I feel deeply
Like a childless butterfly
In the darkness only
My mind is slowly
But my heart truly
Is filled with melancholy

Time:  0:00:15.051003
Standing here all lonely
I can see clearly
And I feel deeply
Like a childless butterfly
In the darkness only
My mind is slowly
But my heart truly
Is filled with melancholy
As if it suddenly
Was born of folly
From which nothing really
Can escape me completely
So as to finally
Let go
Time:  0:00:20.262001
Standing here all lonely
I can see clearly
And I feel deeply
Like a childless butterfly
In the darkness only
My mind is slowly
But my heart truly
Is filled with melancholy
As if it suddenly
Was born of folly
From which nothing really
Can escape me completely
So as to finally
Acknow
Time:  0:00:17.435002
Standing here all lonely
I can see clearly
And I feel deeply
Like a childless butterfly
In the darkness only
My mind is slowly
But my heart truly
Is filled with melancholy
As if it suddenly
Was born of folly
From which nothing really
Can escape me completely
So as to finally
Acknowledge
Time:  0:00:18.925002
Standing here all lonely
I can see clearly
And I feel deep

Time:  0:00:25.257003
Standing here all lonely
I can see clearly
And I feel deeply
Like a childless butterfly
In the darkness only
My mind is slowly
But my heart truly
Is filled with melancholy
As if it suddenly
Was born of folly
From which nothing really
Can escape me completely
So as to finally
Acknowledge that I actually
Am not alonein
Time:  0:00:28.920999
Standing here all lonely
I can see clearly
And I feel deeply
Like a childless butterfly
In the darkness only
My mind is slowly
But my heart truly
Is filled with melancholy
As if it suddenly
Was born of folly
From which nothing really
Can escape me completely
So as to finally
Acknowledge that I actually
Am not aloneYet
Time:  0:00:32.675004
Standing here all lonely
I can see clearly
And I feel deeply
Like a childless butterfly
In the darkness only
My mind is slowly
But my heart truly
Is filled with melancholy
As if it suddenly
Was born of folly
From which nothing really
Can escape me completely
So as to finally
Acknowledge that I 

Time:  0:00:08.734963
Standing here all lonely
I can see clearly
And I feel deeply
Like a childless butterfly
In the darkness only
My mind is slowly
But my heart truly
Is filled with melancholy
As if it suddenly
Was born of folly
From which nothing really
Can escape me completely
So as to finally
Acknowledge that I actually
Am not aloneYet simply
That I am absolutely
Not
Time:  0:00:14.235002
Standing here all lonely
I can see clearly
And I feel deeply
Like a childless butterfly
In the darkness only
My mind is slowly
But my heart truly
Is filled with melancholy
As if it suddenly
Was born of folly
From which nothing really
Can escape me completely
So as to finally
Acknowledge that I actually
Am not aloneYet simply
That I am absolutely
The
Time:  0:00:13.236029
Standing here all lonely
I can see clearly
And I feel deeply
Like a childless butterfly
In the darkness only
My mind is slowly
But my heart truly
Is filled with melancholy
As if it suddenly
Was born of folly
From which nothing rea

Time:  0:00:20.560961
Standing here all lonely
I can see clearly
And I feel deeply
Like a childless butterfly
In the darkness only
My mind is slowly
But my heart truly
Is filled with melancholy
As if it suddenly
Was born of folly
From which nothing really
Can escape me completely
So as to finally
Acknowledge that I actually
Am not aloneYet simply
That I am absolutely
Unable to be wholly
With you
Time:  0:00:23.450000
Standing here all lonely
I can see clearly
And I feel deeply
Like a childless butterfly
In the darkness only
My mind is slowly
But my heart truly
Is filled with melancholy
As if it suddenly
Was born of folly
From which nothing really
Can escape me completely
So as to finally
Acknowledge that I actually
Am not aloneYet simply
That I am absolutely
Unable to be wholly
With any
Time:  0:00:22.195035
Standing here all lonely
I can see clearly
And I feel deeply
Like a childless butterfly
In the darkness only
My mind is slowly
But my heart truly
Is filled with melancholy
As if it

Time:  0:00:30.414536
beam 0: Standing here all lonely
I can see clearly
And I feel deeply
Like a childless butterfly
In the darkness only
My mind is slowly
But my heart truly
Is filled with melancholy
As if it suddenly
Was born of folly
From which nothing really
Can escape me completely
So as to finally
Acknowledge that I actually
Am not aloneYet simply
That I am absolutely
Unable to be wholly
All alone and utterly
With no other family
beam 1: Standing here all lonely
I can see clearly
And I feel deeply
Like a childless butterfly
In the darkness only
My mind is slowly
But my heart truly
Is filled with melancholy
As if it suddenly
Was born of folly
From which nothing really
Can escape me completely
So as to finally
Acknowledge that I actually
Am not aloneYet simply
That I am absolutely
Unable to be wholly
All alone and utterly
With no other earthly
beam 2: Standing here all lonely
I can see clearly
And I feel deeply
Like a childless butterfly
In the darkness only
My mind is slowly
But 

Different result for Repetition penalty 4.0

In [78]:
from transformers import (
    BeamSearchScorer,
    LogitsProcessorList,
    RepetitionPenaltyLogitsProcessor,
    StoppingCriteriaList,
    MaxLengthCriteria
)
import torch

# how many beams to track during the Viterbi algorithm
num_beams = 3
# how many beams to return after the algorithm
num_return_beams = 3

# the prompt to continue
prompt = 'Standing here all lonely'

# tokenizing the prompt
prompt_tokenized = tokenizer(prompt, return_tensors='pt' )
prompt_tokenized = prompt_tokenized['input_ids']

# instantiating a BeamSearchScorer
beam_scorer = BeamSearchScorer(
    batch_size = prompt_tokenized.shape[0],
    num_beams = num_beams,
    num_beam_hyps_to_keep = num_return_beams,
    device=model.device
)

# using our automata to enforce the desired sequence
fsa4 = word_count_rhyme_FSA()
logits_processor = LogitsProcessorList([ConsecutiveLogits(fsa=fsa4, vocab=tokenizer.vocab), RepetitionPenaltyLogitsProcessor(4.0)])

# running beam search using our custom LogitsProcessor
generated = model.beam_search(
    torch.cat([prompt_tokenized] * num_beams),
    beam_scorer,
    logits_processor = logits_processor,
    stopping_criteria=StoppingCriteriaList([MaxLengthCriteria(max_length=100)])
)

# printing the output beams
for index, output_tokenized in enumerate(generated):
  output = tokenizer.decode(output_tokenized)
  print(f'beam {index}: {output}')

Standing here all lonely
Time:  0:00:42.284002
Standing here all lonely
Time:  0:00:47.686996
Standing here all lonely
Time:  0:00:43.598002
Standing here all lonely

Time:  0:00:08.826000
Standing here all lonely

Time:  0:00:08.887000
Standing here all lonely

Time:  0:00:08.777985
Standing here all lonely
I
Time:  0:00:11.327003
Standing here all lonely
The
Time:  0:00:13.135000
Standing here all lonely
A
Time:  0:00:10.758999
Standing here all lonely
I can
Time:  0:00:16.754000
Standing here all lonely
I am
Time:  0:00:16.389042
Standing here all lonely
I don
Time:  0:00:18.047959
Standing here all lonely
I can see
Time:  0:00:23.387002
Standing here all lonely
I can feel
Time:  0:00:23.460030
Standing here all lonely
I am a
Time:  0:00:20.424992
Standing here all lonely
I can see only
Time:  0:00:32.535037
Standing here all lonely
I can see clearly
Time:  0:00:33.016966
Standing here all lonely
I can feel only
Time:  0:00:32.204998
Standing here all lonely
I can see clearly

Time:

Time:  0:00:08.880031
Standing here all lonely
I can see clearly
And I feel deeply
Like a childless butterfly
In the darkness only
My mind is completely

Time:  0:00:08.862000
Standing here all lonely
I can see clearly
And I feel deeply
Like a childless butterfly
In the darkness only
My mind is slowly
But
Time:  0:00:14.022000
Standing here all lonely
I can see clearly
And I feel deeply
Like a childless butterfly
In the darkness only
My mind is slowly
The
Time:  0:00:13.364034
Standing here all lonely
I can see clearly
And I feel deeply
Like a childless butterfly
In the darkness only
My mind is slowly
As
Time:  0:00:12.295000
Standing here all lonely
I can see clearly
And I feel deeply
Like a childless butterfly
In the darkness only
My mind is slowly
As it
Time:  0:00:18.594039
Standing here all lonely
I can see clearly
And I feel deeply
Like a childless butterfly
In the darkness only
My mind is slowly
As if
Time:  0:00:17.607960
Standing here all lonely
I can see clearly
And I feel de

Time:  0:00:38.073001
Standing here all lonely
I can see clearly
And I feel deeply
Like a childless butterfly
In the darkness only
My mind is slowly
As if it suddenly
Aching for me desperately
It feels like finally

Time:  0:00:08.815999
Standing here all lonely
I can see clearly
And I feel deeply
Like a childless butterfly
In the darkness only
My mind is slowly
As if it suddenly
Aching for me desperately
To find my family

Time:  0:00:08.777042
Standing here all lonely
I can see clearly
And I feel deeply
Like a childless butterfly
In the darkness only
My mind is slowly
As if it suddenly
Aching for me desperately
To find myself safely
Time:  0:00:40.247960
Standing here all lonely
I can see clearly
And I feel deeply
Like a childless butterfly
In the darkness only
My mind is slowly
As if it suddenly
Aching for me desperately
It feels like finally
The
Time:  0:00:13.347003
Standing here all lonely
I can see clearly
And I feel deeply
Like a childless butterfly
In the darkness only
My mind

Time:  0:00:13.631999
Standing here all lonely
I can see clearly
And I feel deeply
Like a childless butterfly
In the darkness only
My mind is slowly
As if it suddenly
Aching for me desperately
It feels like finally
The time has actually
Now come to fly
With
Time:  0:00:15.264963
Standing here all lonely
I can see clearly
And I feel deeply
Like a childless butterfly
In the darkness only
My mind is slowly
As if it suddenly
Aching for me desperately
It feels like finally
The time has actually
Now come to truly
Let go
Time:  0:00:19.322041
Standing here all lonely
I can see clearly
And I feel deeply
Like a childless butterfly
In the darkness only
My mind is slowly
As if it suddenly
Aching for me desperately
It feels like finally
The time has actually
Now come to fly
With my
Time:  0:00:22.189998
Standing here all lonely
I can see clearly
And I feel deeply
Like a childless butterfly
In the darkness only
My mind is slowly
As if it suddenly
Aching for me desperately
It feels like finally
The 

Time:  0:00:13.745034
Standing here all lonely
I can see clearly
And I feel deeply
Like a childless butterfly
In the darkness only
My mind is slowly
As if it suddenly
Aching for me desperately
It feels like finally
The time has actually
Now come to truly
Let go and simply
Leave this world completely
So
Time:  0:00:13.184952
Standing here all lonely
I can see clearly
And I feel deeply
Like a childless butterfly
In the darkness only
My mind is slowly
As if it suddenly
Aching for me desperately
It feels like finally
The time has actually
Now come to truly
Let go and simply
Leave this world completely
So that
Time:  0:00:21.121037
Standing here all lonely
I can see clearly
And I feel deeply
Like a childless butterfly
In the darkness only
My mind is slowly
As if it suddenly
Aching for me desperately
It feels like finally
The time has actually
Now come to truly
Let go and simply
Leave this world completely
For now
Time:  0:00:21.585036
Standing here all lonely
I can see clearly
And I feel de

Time:  0:00:08.655999
Standing here all lonely
I can see clearly
And I feel deeply
Like a childless butterfly
In the darkness only
My mind is slowly
As if it suddenly
Aching for me desperately
It feels like finally
The time has actually
Now come to truly
Let go and simply
Leave this world completely
So that we really
Can live together happily

Time:  0:00:08.763033
Standing here all lonely
I can see clearly
And I feel deeply
Like a childless butterfly
In the darkness only
My mind is slowly
As if it suddenly
Aching for me desperately
It feels like finally
The time has actually
Now come to truly
Let go and simply
Leave this world completely
So that we really
Can live together freely

Time:  0:00:08.779921
Standing here all lonely
I can see clearly
And I feel deeply
Like a childless butterfly
In the darkness only
My mind is slowly
As if it suddenly
Aching for me desperately
It feels like finally
The time has actually
Now come to truly
Let go and simply
Leave this world completely
So that 

Time:  0:00:21.136034
Standing here all lonely
I can see clearly
And I feel deeply
Like a childless butterfly
In the darkness only
My mind is slowly
As if it suddenly
Aching for me desperately
It feels like finally
The time has actually
Now come to truly
Let go and simply
Leave this world completely
So that we really
Can live together peacefully
No matter how ugly
We may be
Time:  0:00:24.241997
Standing here all lonely
I can see clearly
And I feel deeply
Like a childless butterfly
In the darkness only
My mind is slowly
As if it suddenly
Aching for me desperately
It feels like finally
The time has actually
Now come to truly
Let go and simply
Leave this world completely
So that we really
Can live together peacefully
No matter how ugly
We might be
Time:  0:00:25.870001
Standing here all lonely
I can see clearly
And I feel deeply
Like a childless butterfly
In the darkness only
My mind is slowly
As if it suddenly
Aching for me desperately
It feels like finally
The time has actually
Now com

Time:  0:00:36.676964
Standing here all lonely
I can see clearly
And I feel deeply
Like a childless butterfly
In the darkness only
My mind is slowly
As if it suddenly
Aching for me desperately
It feels like finally
The time has actually
Now come to truly
Let go and simply
Leave this world completely
So that we really
Can live together peacefully
No matter how ugly
We may be individually
Or even as family

Time:  0:00:08.823099
Standing here all lonely
I can see clearly
And I feel deeply
Like a childless butterfly
In the darkness only
My mind is slowly
As if it suddenly
Aching for me desperately
It feels like finally
The time has actually
Now come to truly
Let go and simply
Leave this world completely
So that we really
Can live together peacefully
No matter how ugly
We may be individually
But in our family

Time:  0:00:08.761998
Standing here all lonely
I can see clearly
And I feel deeply
Like a childless butterfly
In the darkness only
My mind is slowly
As if it suddenly
Aching for me d

Time:  0:00:13.697999
Standing here all lonely
I can see clearly
And I feel deeply
Like a childless butterfly
In the darkness only
My mind is slowly
As if it suddenly
Aching for me desperately
It feels like finally
The time has actually
Now come to truly
Let go and simply
Leave this world completely
So that we really
Can live together peacefully
No matter how ugly
We may be individually
Or even as family
Just because you probably
Don
Time:  0:00:13.089040
beam 0: Standing here all lonely
I can see clearly
And I feel deeply
Like a childless butterfly
In the darkness only
My mind is slowly
As if it suddenly
Aching for me desperately
It feels like finally
The time has actually
Now come to truly
Let go and simply
Leave this world completely
So that we really
Can live together peacefully
No matter how ugly
We may be individually
Or even as family
Just because you probably
Have no
beam 1: Standing here all lonely
I can see clearly
And I feel deeply
Like a childless butterfly
In the darkness 

Using the intersection of two fsas:

In [20]:
from transformers import (
    BeamSearchScorer,
    LogitsProcessorList,
    RepetitionPenaltyLogitsProcessor,
    StoppingCriteriaList,
    MaxLengthCriteria
)
import torch

# how many beams to track during the Viterbi algorithm
num_beams = 10
# how many beams to return after the algorithm
num_return_beams = 10

# the prompt to continue
prompt = 'Standing here so lonely'

# tokenizing the prompt
prompt_tokenized = tokenizer(prompt, return_tensors='pt' )
prompt_tokenized = prompt_tokenized['input_ids']

# instantiating a BeamSearchScorer
beam_scorer = BeamSearchScorer(
    batch_size = prompt_tokenized.shape[0],
    num_beams = num_beams,
    num_beam_hyps_to_keep = num_return_beams,
    device=model.device
)

# using our automata to enforce the desired sequence
rhyme_fsa = word_count_rhyme_FSA()
even_fsa = even_FSA()
intersect_fsa = rhyme_fsa.intersect(even_fsa).trim()
logits_processor = LogitsProcessorList([ConsecutiveLogits(fsa=intersect_fsa, vocab=tokenizer.vocab), RepetitionPenaltyLogitsProcessor(3.0)])

# running beam search using our custom LogitsProcessor
generated = model.beam_search(
    torch.cat([prompt_tokenized] * num_beams),
    beam_scorer,
    logits_processor = logits_processor,
    stopping_criteria=StoppingCriteriaList([MaxLengthCriteria(max_length=50)])
)

# printing the output beams
for index, output_tokenized in enumerate(generated):
  output = tokenizer.decode(output_tokenized)
  print(f'beam {index}: {output}')

Standing here so lonely
Time:  0:00:37.071126
Standing here so lonely
Time:  0:00:36.491446
Standing here so lonely
Time:  0:00:36.916649
Standing here so lonely
Time:  0:00:37.209315
Standing here so lonely
Time:  0:00:40.278655
Standing here so lonely
Time:  0:00:36.780201
Standing here so lonely
Time:  0:00:36.915575
Standing here so lonely
Time:  0:00:36.618556
Standing here so lonely
Time:  0:00:36.775364
Standing here so lonely
Time:  0:00:37.226657
Standing here so lonely

Time:  0:00:12.452487
Standing here so lonely

Time:  0:00:10.386832
Standing here so lonely

Time:  0:00:10.322783
Standing here so lonely

Time:  0:00:09.999509
Standing here so lonely

Time:  0:00:10.410841
Standing here so lonely

Time:  0:00:10.312329
Standing here so lonely

Time:  0:00:10.207064
Standing here so lonely

Time:  0:00:10.087645
Standing here so lonely

Time:  0:00:10.173687
Standing here so lonely

Time:  0:00:10.082144
Standing here so lonely
It
Time:  0:00:14.388203
Standing here so lone

Time:  0:00:16.317548
Standing here so lonely
This is my family
My name is Lily
If
Time:  0:00:13.386352
Standing here so lonely
This is my family
My name is Kimberly
So
Time:  0:00:14.943132
Standing here so lonely
This is my family
My name is Lily
Just
Time:  0:00:17.678350
Standing here so lonely
This is my family
My name is Lily
We have
Time:  0:00:22.475346
Standing here so lonely
This is my family
My name is Kimberly
We have
Time:  0:00:22.513555
Standing here so lonely
This is my family
My name is Lily
What do
Time:  0:00:23.130054
Standing here so lonely
This is my family
My name is Lily
Just like
Time:  0:00:25.411956
Standing here so lonely
This is my family
My name is Lily
We were
Time:  0:00:23.557646
Standing here so lonely
This is my family
My name is Kimberly
We live
Time:  0:00:22.938813
Standing here so lonely
This is my family
My name is Lily
We live
Time:  0:00:22.920016
Standing here so lonely
This is my family
My name is Lily
We love
Time:  0:00:23.277483
Standing 

Time:  0:00:30.568229
Standing here so lonely
This is my family
My name is Kimberly
We live in Philly
So we have really
Time:  0:00:30.610764
Standing here so lonely
This is my family
My name is Kimberly
We live in Philly
So we have probably
Time:  0:00:32.388215
Standing here so lonely
This is my family
My name is Kimberly
We live in Philly
Here we goin
Time:  0:00:30.175745
Standing here so lonely
This is my family
My name is Lily
We were born peacefully
In this land only

Time:  0:00:09.875017
Standing here so lonely
This is my family
My name is Kimberly
We live in Philly
Here we goin only
Time:  0:00:30.320516
Standing here so lonely
This is my family
My name is Kimberly
We live in Philly
So we know only

Time:  0:00:10.080282
Standing here so lonely
This is my family
My name is Kimberly
We live in Philly
Here we goin slowly
Time:  0:00:34.133943
Standing here so lonely
This is my family
My name is Kimberly
We live in Philly
Here we goinward
Time:  0:00:35.543556
Standing here so l

Time:  0:00:10.594751
Standing here so lonely
This is my family
My name is Lily
We were born peacefully
In this land only
When we were really

Time:  0:00:10.631631
Standing here so lonely
This is my family
My name is Lily
We were born peacefully
In this land only
To live together freely
So
Time:  0:00:14.895056
Standing here so lonely
This is my family
My name is Lily
We were born peacefully
In this land only
To live together freely
It
Time:  0:00:15.488219
Standing here so lonely
This is my family
My name is Lily
We were born peacefully
In this land only
To live together freely
No
Time:  0:00:15.885060
Standing here so lonely
This is my family
My name is Lily
We were born peacefully
In this land only
To live together freely
That
Time:  0:00:17.588855
Standing here so lonely
This is my family
My name is Lily
We were born peacefully
In this land only
To live together freely
With
Time:  0:00:18.136728
Standing here so lonely
This is my family
My name is Lily
We were born peacefully
In t

Time:  0:00:14.488780
Standing here so lonely
This is my family
My name is Lily
We were born peacefully
In this land only
To live together freely
Together we will eventually
When
Time:  0:00:16.738180
Standing here so lonely
This is my family
My name is Lily
We were born peacefully
In this land only
To live together freely
Together we will eventually
As
Time:  0:00:13.949823
Standing here so lonely
This is my family
My name is Lily
We were born peacefully
In this land only
To live together freely
Together we will eventually
That
Time:  0:00:16.589292
Standing here so lonely
This is my family
My name is Lily
We were born peacefully
In this land only
To live together freely
Together we will eventually
So
Time:  0:00:15.209663
Standing here so lonely
This is my family
My name is Lily
We were born peacefully
In this land only
To live together freely
Together we will eventually
If
Time:  0:00:13.472583
Standing here so lonely
This is my family
My name is Lily
We were born peacefully
In this

Time:  0:00:09.787815
Standing here so lonely
This is my family
My name is Lily
We were born peacefully
In this land only
To live together freely
Together we will eventually
Come to loveheretogether
Time:  0:00:43.959450
Standing here so lonely
This is my family
My name is Lily
We were born peacefully
In this land only
To live together freely
Together we will eventually
Come to lovehere safely
Time:  0:00:35.785349
Standing here so lonely
This is my family
My name is Lily
We were born peacefully
In this land only
To live together freely
Together we will eventually
Come to loveherefore
Time:  0:00:39.296484
Standing here so lonely
This is my family
My name is Lily
We were born peacefully
In this land only
To live together freely
Together we will eventually
Come to loveherewith
Time:  0:00:40.322662
Standing here so lonely
This is my family
My name is Lily
We were born peacefully
In this land only
To live together freely
Together we will eventually
Come to loveherehome
Time:  0:00:40.118

Time:  0:00:35.948249
Standing here so lonely
This is my family
My name is Lily
We were born peacefully
In this land only
To live together freely
Together we will eventually
Come to rest safely
As long as Heavenly
Time:  0:00:33.115319
Standing here so lonely
This is my family
My name is Lily
We were born peacefully
In this land only
To live together freely
Together we will eventually
Come to rest safely
No matter what really
Time:  0:00:35.519571
Standing here so lonely
This is my family
My name is Lily
We were born peacefully
In this land only
To live together freely
Together we will eventually
Come to rest safely
Just like that softly
Time:  0:00:36.804030
Standing here so lonely
This is my family
My name is Lily
We were born peacefully
In this land only
To live together freely
Together we will eventually
Come to rest safely
Just like your Family
Time:  0:00:39.435176
Standing here so lonely
This is my family
My name is Lily
We were born peacefully
In this land only
To live together

Time:  0:00:31.285796
Standing here so lonely
This is my family
My name is Lily
We were born peacefully
In this land only
To live together freely
Together we will eventually
Come to rest safely
Just like your family
So what do
Time:  0:00:29.599832
Standing here so lonely
This is my family
My name is Lily
We were born peacefully
In this land only
To live together freely
Together we will eventually
Come to rest safely
Just like your family
So many people
Time:  0:00:35.152090
Standing here so lonely
This is my family
My name is Lily
We were born peacefully
In this land only
To live together freely
Together we will eventually
Come to rest safely
Just like your family
So many things
Time:  0:00:35.101533
Standing here so lonely
This is my family
My name is Lily
We were born peacefully
In this land only
To live together freely
Together we will eventually
Come to rest safely
Just like your family
So many of
Time:  0:00:29.531747
Standing here so lonely
This is my family
My name is Lily
We w